## PV01

The *PV01* for a plain vanilla swap can be calculated as

$$PV01 = 0.01\% \sum_{i=\alpha +1}^{\beta} \tau_i P(0, T_i)$$

In [1]:
import numpy as np

from dateutil.relativedelta import relativedelta

def TimeInterval(interval):
    tag = interval[-1].lower()
    value = int(interval[:-1])
    if tag == "d":
        return relativedelta(days=value)
    elif tag == "m":
        return relativedelta(months=value)
    elif tag == "y":
        return relativedelta(years=value)
    else:
        raise ValueError(f"Unable to convert {interval}, probably wrong units.")
    
def generate_dates(start_date, end_date, frequency="1y"):
    if isinstance(end_date, str):
        end_date = start_date + TimeInterval(end_date)
    d = start_date
    dates = [start_date]
    while True:
        d += TimeInterval(frequency)
        if d < end_date:
            dates.append(d)
        else:
            dates.append(end_date)
            break
    return dates

class DiscountCurve:
  def __init__(self, pillar_dates, discount_factors):
    self.discount_factors = np.array(discount_factors)
    self.pillar_dates = pillar_dates

  def df(self, adate):
    pillars = [p.toordinal() for p in self.pillar_dates]
    return np.interp(adate.toordinal(), pillars, self.discount_factors)

In [2]:
class InterestRateSwap:
    def __init__(self, nominal, start_date, maturity, fixed_rate, frequency_fix="12m", side=1):
        self.N = nominal
        self.K = fixed_rate
        self.dates = generate_dates(start_date, maturity, frequency_fix)
        self.side = side

    def npv(self, dc):
      A = self.annuity(dc)
      return self.N*(self.K*A - dc.df(self.dates[0]) + dc.df(self.dates[-1]))

    def swap_rate_single_curve(self, dc):
        den = 0
        num = dc.df(self.dates[0]) - dc.df(self.dates[-1])
        for i in range(1, len(self.dates)):
            tau = (self.dates[i]-self.dates[i-1]).days/360
            den += dc.df(self.dates[i])*tau
        return num/den

    def annuity(self, dc):
        a = 0
        for i in range(1, len(self.dates)):
            tau = (self.dates[i]-self.dates[i-1]).days/360
            a += tau*dc.df(self.dates[i])
        return a

    def pv01(self, dc):
        return 0.0001*self.annuity(dc)


In [4]:
from datetime import date
from dateutil.relativedelta import relativedelta

n = 6
today = date.today()
dates = [today+relativedelta(years=i) for i in range(n)]
dfs = [1/(1+0.05)**i for i in range(n)]
dc = DiscountCurve(dates, dfs)

irs = InterestRateSwap(1, today, "5y", 0.055, "3m")

print ("IRS PV01: {:.5f}".format(irs.pv01(dc)))

IRS PV01: 0.00045


Since each vanilla swap can be considered as an exchange of a *fixed rate bond* and a *floater*, deltas can be computed using the same "instruments" as bonds.
In particular it can be used the *duration* which estimates the change in the bond price by the change in its yield

$$\frac{\Delta P}{P} = - D\Delta y$$

For example, if a bond has a duration of 5, then a 1% increase in interest rates would cause the bond's price to fall by 5%.
Consider that to obtain previous equation a first order Taylor series expantion has been used, that's why it holds only for small rate variation (otherwise convexity has to be considered).


Once duration has been computed $PV01$ can be estimated through the following approximation

$$PV01 = D \cdot P_{\text{fix}} \cdot 0.01\%$$

## DV01

One way to compute DV01, is by manually shift the interest rate curve, compute the contract price twice and check the variation.

In [6]:
import numpy as np

from datetime import date
from dateutil.relativedelta import relativedelta

n = 6
today = date.today()
dates = [today + relativedelta(years=i) for i in range(n)]

dr = 0.0001
rates_up = np.array([0.05]*n) + dr
rates_down = np.array([0.05]*n) - dr
dfs_up = [1/(1+rates_up[i])**i for i in range(n)]
dfs_down = [1/(1+rates_down[i])**i for i in range(n)]
dc_up = DiscountCurve(dates, dfs_up)
dc_down = DiscountCurve(dates, dfs_down)

irs = InterestRateSwap(1, today, "5y", 0.055, "3m")
dv01 = (irs.npv(dc_down) - irs.npv(dc_up))/2
print (f"IRS DV01: {dv01:.5f}")

IRS DV01: 0.00043


\begin{equation}
\textbf{DV01} = \underbrace{-\sum_{j}\tau_jP_j}_{\text{PV01}}+\underbrace{\sum_{j}\left(K\sum_{i=\alpha+1}^\beta\tau_i\frac{\partial P_i}{\partial r_j} - \sum_{k=\alpha+1}^\beta L_k\tau_k\frac{\partial P_k}{\partial r_j}\right)}_{\text{additional terms}}
\end{equation}

## Algorithmic Differientation

Consider the following function
\begin{equation}
\begin{cases}
\text{Function: } f(x_0, x_1) = 2x_0^2 + 3x_1\\
\text{Solution: } \frac{df}{dx_0} = 4x_0, \text{ and } \frac{df}{dx_1}=3
\end{cases}
\end{equation}

Compute the AD both using tangent and adjoint technique, when $x_0=2$ and $x_1=3$,
\begin{equation}
\frac{df}{dx_0} = 8, \text{ and } \frac{df}{dx_1}=3
\end{equation}


In [7]:
import tensorflow as tf

def f(x):
  return 2*x[0]**2+3*x[1]

def df_tangent(x):
  return 4*x[0], 3

def df_adjoint(x):
  x = tf.Variable(x, dtype='float', name='x')
  with tf.GradientTape() as tape:
    f = 2*x[0]**2+3*x[1]
  return tape.gradient(f, x)

x = (2, 3)
print (f"f(x) = {f(x)}")
print (f"tangent = {df_tangent(x)}")
print (f"aad = {df_adjoint(x)}")

f(x) = 17
tangent = (8, 3)
aad = [8. 3.]


Consider a 5-years receiver Interest Rate Swap with a 1M notional, exchanging a fixed rate of 5\% with a flat 1\% LIBOR rate with annual payments.
Compute DV01 with and without algorithmic differentiation. Compare the results.

Below the two differentiation modes have been implemented.
For the "tangent" mode the explicit derivatives have been computed, as an example (remember that $F_i=\frac{r_i t_i - r_{i-1} t_{i-1}}{\tau}$):
$$
\cfrac{\partial PV_{float}}{\partial r_i} = \cfrac{\partial}{\partial r_i}\sum_j N\tau F_j e^{-r_j t_j} = N\tau \Bigg[ t_i - F_i t_i - t_{i-1}\Bigg]e^{-r_i t_i}
$$

$$
\cfrac{\partial PV_{fix}}{\partial r_i} = \cfrac{\partial}{\partial r_i}\sum_i N\tau K e^{-r_i t_i} = - N\tau K t_i e^{-r_i t_i}
$$

In [54]:
import numpy as np
import tensorflow as tf

class Swap:
  def __init__(self, notional, fixed_rate, tau, terms, rates):
    self.N = notional
    self.K = fixed_rate
    self.tau = tau
    self.terms = np.array(terms)
    self.rates = np.array(rates)

  def swap_price(self, dr=0.0001):
    rates = tf.Variable(self.rates, name="rates", dtype=tf.float64)
    fixed_pv = tf.Variable(0.0, dtype=tf.float64)
    float_pv = tf.Variable(0.0, dtype=tf.float64)

    with tf.GradientTape(persistent=True) as tape:
      fixed_pv = fixed_pv + self.N*self.K*self.tau*tf.reduce_sum(tf.math.exp(-rates[1:]*self.terms[1:]))
      for j in range(1, len(self.terms)):
        float_pv = float_pv + self.N*(rates[j]*self.terms[j]-rates[j-1]*self.terms[j-1])*tf.math.exp(-rates[j]*self.terms[j])

    fixed_pv_dot = np.sum(dr*tape.gradient(fixed_pv, rates))
    float_pv_dot = np.sum(dr*tape.gradient(float_pv, rates))

    swap_pv = (fixed_pv - float_pv)
    swap_pv_dot = (fixed_pv_dot - float_pv_dot)

    return swap_pv, swap_pv_dot

  def swap_price_tangent_mode_manual(self, r_dot=0.0001):
    fixed_pv = 0.0
    fixed_pv_dot = 0.0

    fixed_pv = self.N*self.K*self.tau*np.exp(-self.rates[1:]*self.terms[1:]).sum()
    for i in range(1, len(self.terms)):
        fixed_pv_dot += -self.terms[i]*self.N*self.K*self.tau*np.exp(-self.rates[i]*self.terms[i])*r_dot
    
    float_pv = 0.0
    float_pv_dot = 0.0
    for j in range(1, len(self.terms)):
        F = (self.rates[j]*self.terms[j]-self.rates[j-1]*self.terms[j-1])/self.tau
        float_pv += self.N*F*self.tau*np.exp(-self.rates[j]*self.terms[j])
        float_pv_dot += -self.terms[j-1]*self.N*self.tau*np.exp(-self.rates[j]*self.terms[j])*r_dot
        float_pv_dot += self.terms[j]*self.N*self.tau*np.exp(-self.rates[j]*self.terms[j])*(1-F)*r_dot
    swap_pv = (fixed_pv - float_pv)
    swap_pv_dot = (fixed_pv_dot - float_pv_dot)

    return swap_pv, swap_pv_dot

In [56]:

N = 1e6
fixed_rate  = 0.015
tau         = 1.0
terms       = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])
rates       = np.array([0.01, 0.012, 0.013, 0.014, 0.016, 0.017])

swap = Swap(N, fixed_rate, tau, terms, rates)
price, dv01 = swap.swap_price()

print (f"Swap price: {price:,.2f} ({swap.swap_price_tangent_mode_manual()[0]:,.2f})")
print (f"DV01: {dv01:,.2f} ({swap.swap_price_tangent_mode_manual()[1]:,.2f})")

Swap price: -9,097.43 (-9,097.43)
DV01: -472.60 (-472.60)
